In [ ]:
!pip install transformers
!pip install konlpy

In [ ]:
import torch
import transformers
from transformers import AutoModelWithLMHead, PreTrainedTokenizerFast
from fastai.text.all import *
import fastai
import re
import os
import json

# finetuned KoGPT download

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Euna9/kogpt2_ku_2")
model = AutoModelForCausalLM.from_pretrained("Euna9/kogpt2_ku_2")

# 주제 추천(postrank)

In [ ]:
# 원하는 시기의 날짜를 yyyy-mm-dd 형식으로 입력해주세요
!python postrank_word_1.0.py 2022-08-01

In [ ]:
words = pd.read_csv('text_rank.csv')
words.columns = ['date', 'word', 'rank']

# 주제 키워드로 글 생성

In [ ]:
# [주제 키워드+조사] 형식을 prompts에 입력해주세요
prompts = '에타가'
prompt_ids = tokenizer.encode(prompts)
inp = tensor(prompt_ids)[None].cuda() # cuda에러 발생할 경우 -> .cuda()제거
preds = model.generate(inp,
                        max_length=128,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        repetition_penalty=2.0,
                        use_cache=True
                      )
generated = tokenizer.decode(preds[0].cpu().numpy())
print(generated)

## TextStyleTransfer

In [ ]:
os.chdir('TextStyleTransfer_supervised')
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/AIKU/TextStyleTransfer_supervised'

In [ ]:
import argparse
import torch
from transformers import DataCollatorForSeq2Seq, BartForConditionalGeneration
from models.model import *
from utils.dataset import *
import pandas as pd

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
if __name__ == '__main__':
    PATH = "checkpoints"
    tokenizer = StyleTransferTokenizer
    model=BartForConditionalGeneration.from_pretrained(PATH, local_files_only=True)
    BartCollator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model
    )

    sample_text = [
    "에타가 뭔지 모르겠어",
    "그냥 그냥 저런거만 하고싶어서 쓴 글임"
    ]

    encoder_text = list(map(lambda x: f"<informal>{x}", sample_text))
    batch = tokenizer(encoder_text, max_length=64, truncation=True, padding='max_length')

    input = BatchEncoding({
      'input_ids':torch.tensor(batch['input_ids']),
      'decoder_input_ids': torch.tensor(list(map(lambda x: [x[0]], batch['input_ids'])))
      })

    generated_ids = model.generate(**input, num_beams=4, min_length=5, max_length=64)
    res = tokenizer.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    print("+-----------------------------------------------------------------------------------+")
    for i in range(len(encoder_text)):
        print(f"| {encoder_text[i]}")
        print(f"| {'결과:':>5}{res[i]} ")
        print("+-----------------------------------------------------------------------------------+")